In [1]:
%store -r df

In [2]:
df

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",1.2,105,бензин,74000,2013.0,OCTAVIA,5.0,2014,ROBOT,3.0,0,ORIGINAL,передний,LEFT,-1.0
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110,бензин,60563,2017.0,OCTAVIA,5.0,2017,MECHANICAL,1.0,0,ORIGINAL,передний,LEFT,-1.0
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",1.8,152,бензин,88000,2013.0,SUPERB,5.0,2014,ROBOT,1.0,0,ORIGINAL,передний,LEFT,-1.0
3,лифтбек,SKODA,коричневый,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110,бензин,95000,2013.0,OCTAVIA,5.0,2014,AUTOMATIC,1.0,0,ORIGINAL,передний,LEFT,-1.0
4,лифтбек,SKODA,белый,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152,бензин,58536,2008.0,OCTAVIA,5.0,2012,AUTOMATIC,1.0,0,ORIGINAL,передний,LEFT,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121944,минивэн,SSANG_YONG,чёрный,СВЕТИТ КРУТИТ РАБОТАЕТ ВСЁ. БЕЗ ТОРГА .В МОСК...,2.7,163,дизель,260000,2004.0,RODIUS,5.0,2006,AUTOMATIC,3.0,0,ORIGINAL,полный,LEFT,450000.0
121945,минивэн,SSANG_YONG,серебристый,"Автомобиль на ходу, каждый день в движении.\nС...",2.7,163,дизель,300000,2004.0,RODIUS,5.0,2007,AUTOMATIC,3.0,1,ORIGINAL,полный,LEFT,485000.0
121946,минивэн,SSANG_YONG,чёрный,"Автомобиль куплен в декабре 2015 г., новым. Вс...",2.0,149,дизель,126000,2013.0,STAVIC,5.0,2014,AUTOMATIC,1.0,1,ORIGINAL,полный,LEFT,1090000.0
121947,минивэн,SSANG_YONG,серый,Продаю семейного любимца. Большой вместительны...,2.0,149,дизель,84000,2013.0,STAVIC,5.0,2013,AUTOMATIC,3.0,1,ORIGINAL,полный,LEFT,1180000.0
